In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'deepfake-and-real-images' dataset.
Path to dataset files: /kaggle/input/deepfake-and-real-images


In [2]:
!pip install -q timm tqdm

In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import timm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from torch.amp import autocast, GradScaler

# cuDNN autotune
torch.backends.cudnn.benchmark = True

# Device & AMP scaler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

scaler = GradScaler()

Using device: cuda


In [4]:
class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.samples = []
        classes = sorted(
            d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d))
        )
        self.class_to_idx = {cls: i for i, cls in enumerate(classes)}
        for cls in classes:
            folder = os.path.join(root_dir, cls)
            for fname in os.listdir(folder):
                path = os.path.join(folder, fname)
                if os.path.isfile(path) and fname.lower().endswith(('.jpg','jpeg','.png')):
                    self.samples.append((path, self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

In [5]:
dataset_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Train'

# Quick sanity check
print("Classes:", os.listdir(dataset_dir))

# Transforms (224×224 required by ViT)
train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225)),
])
val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225)),
])

# Build & split dataset
full_ds = FaceDataset(dataset_dir, transform=train_tfms)
val_size = int(0.2 * len(full_ds))
train_ds, val_ds = random_split(
    full_ds,
    [len(full_ds) - val_size, val_size],
    generator=torch.Generator().manual_seed(42)
)
val_ds.dataset.transform = val_tfms

# Quick-train mode for prototyping
QUICK_TRAIN = False
TRAIN_FRACTION = 0.5  # use 50% of train split
if QUICK_TRAIN:
    reduced = int(len(train_ds) * TRAIN_FRACTION)
    train_ds, _ = random_split(
        train_ds,
        [reduced, len(train_ds) - reduced],
        generator=torch.Generator().manual_seed(1)
    )
    print(f"[Quick-train] using {len(train_ds)} / {len(full_ds)-val_size} samples")

# DataLoader params
BATCH_SIZE = 64
NUM_WORKERS = 4

train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=True
)
val_loader = DataLoader(
    val_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)
print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")

Classes: ['Fake', 'Real']
Train batches: 1751, Val batches: 438


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
model = timm.create_model('vit_tiny_patch16_224', pretrained=True, num_classes=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.1, patience=2
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

In [7]:
num_epochs = 10
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

for epoch in range(1, num_epochs + 1):
    # — Train —
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs} [Train]", leave=False)
    for imgs, labels in train_bar:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            logits = model(imgs)
            loss = criterion(logits, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * imgs.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)
        train_bar.set_postfix(loss=running_loss/total, acc=correct/total)

    train_loss = running_loss / total
    train_acc = correct / total

    # — Validate —
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_probs = []
    all_labels = []
    val_bar = tqdm(val_loader, desc=f"Epoch {epoch}/{num_epochs} [Val]  ", leave=False)
    with torch.no_grad():
        for imgs, labels in val_bar:
            imgs, labels = imgs.to(device), labels.to(device)
            with autocast(device_type='cuda'):
                logits = model(imgs)
                loss = criterion(logits, labels)

            val_loss += loss.item() * imgs.size(0)
            preds = logits.argmax(dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += imgs.size(0)

            probs = torch.softmax(logits, dim=1)[:, 1]
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            val_bar.set_postfix(loss=val_loss/val_total, acc=val_correct/val_total)

    val_loss /= val_total
    val_acc = val_correct / val_total

    # Scheduler & logging
    scheduler.step(val_loss)
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_acc'].append(train_acc)
    history['val_acc'].append(val_acc)

    print(f"Epoch {epoch}/{num_epochs} → "
          f"Train: loss={train_loss:.4f}, acc={train_acc:.4f} | "
          f"Val:   loss={val_loss:.4f}, acc={val_acc:.4f}")

Epoch 1/10 [Train]:   0%|          | 0/1751 [00:00<?, ?it/s]

Epoch 1/10 [Val]  :   0%|          | 0/438 [00:00<?, ?it/s]

Epoch 1/10 → Train: loss=0.1630, acc=0.9290 | Val:   loss=0.1025, acc=0.9597


Epoch 2/10 [Train]:   0%|          | 0/1751 [00:00<?, ?it/s]

Epoch 2/10 [Val]  :   0%|          | 0/438 [00:00<?, ?it/s]

...

Epoch 9/10 [Val]  :   0%|          | 0/438 [00:00<?, ?it/s]

Epoch 9/10 → Train: loss=0.0179, acc=0.9927 | Val:   loss=0.0691, acc=0.9790


Epoch 10/10 [Train]:   0%|          | 0/1751 [00:00<?, ?it/s]

Epoch 10/10 [Val]  :   0%|          | 0/438 [00:00<?, ?it/s]

Epoch 10/10 → Train: loss=0.0101, acc=0.9958 | Val:   loss=0.0804, acc=0.9789


In [8]:
torch.save(model.state_dict(), 'model_weights.pth')

In [9]:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44284 sha256=9aae5c3759065eaf6a06c4ab915275361c988b39725b59360dc1c409ded2067d
  Stored in directory: /root/.cache/pip/wheels/fb/3b/09/2afc520f3d69bc26ae6bd87416759c820a3f7d05c1a077bbf6
Successfully built grad-cam


In [10]:
from pytorch_grad_cam import GradCAM

In [25]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2
import random

def vit_reshape_transform(tensor, height=14, width=14):
    result = tensor[:, 1:, :].reshape(tensor.size(0), height, width, tensor.size(-1))
    result = result.permute(0, 3, 1, 2)  #shape: [B, C, H, W]
    return result

target_layer = model.blocks[-1].norm1
grad_cam = GradCAM(
    model=model,
    target_layers=[target_layer],
    reshape_transform=vit_reshape_transform
)

#picking random from val_loader to test
inputs, labels = next(iter(val_loader))
random_idx = random.randint(0, inputs.shape[0] - 1)

img_tensor = inputs[random_idx].unsqueeze(0).to(device)  # [1, 3, 224, 224]
label = labels[random_idx].item()

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

img = img_tensor.squeeze(0).cpu().numpy().transpose(1, 2, 0)
img = (img * std + mean)
img = np.clip(img, 0, 1)

targets = [ClassifierOutputTarget(label)]
mask = grad_cam(input_tensor=img_tensor, targets=targets)[0]

cam_image = show_cam_on_image(img, mask, use_rgb=True)

cv2.imwrite('test.jpg', cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR))

print(f"grad-worked: {label}")


Grad-CAM applied to sample with label: 0


## inference


In [ ]:
import torch
import timm
import torch.nn as nn
import numpy as np
import cv2
from torchvision import transforms
from PIL import Image

# Grad-CAM imports
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Load model ---
model_infer = timm.create_model('vit_tiny_patch16_224', pretrained=False, num_classes=2)
model_infer.load_state_dict(torch.load('model_weights.pth', map_location=device))
model_infer.to(device)
model_infer.eval()

# --- Preprocess a test image ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225)),
])

img_path = "test.jpg"   # your test image path
pil_img = Image.open(img_path).convert('RGB')
img_tensor = transform(pil_img).unsqueeze(0).to(device)

# For visualization: denormalize to [0,1] RGB
img_for_cam = np.array(pil_img.resize((224,224))) / 255.0

# For transformation: preprocessing image
def vit_reshape_transform(tensor, height=14, width=14):
    result = tensor[:, 1:, :].reshape(tensor.size(0), height, width, tensor.size(-1))
    result = result.permute(0, 3, 1, 2)  #shape: [B, C, H, W]
    return result

# --- Prediction ---
logits = model_infer(img_tensor)
probs = torch.softmax(logits, dim=1)
predicted_class = logits.argmax(dim=1).item()
print(f"Predicted class: {predicted_class} | probs: {probs.detach().cpu().numpy()}")

# --- Grad-CAM setup ---
target_layers = [model_infer.blocks[-1].norm1]   # last norm in final block
targets = [ClassifierOutputTarget(predicted_class)]

with GradCAM(model=model_infer, target_layers=target_layers, reshape_transform=vit_reshape_transform) as cam:
    grayscale_cam = cam(input_tensor=img_tensor, targets=targets)[0, :]

# --- Overlay heatmap ---
cam_image = show_cam_on_image(img_for_cam, grayscale_cam, use_rgb=True)
cv2.imwrite("gradcam_result.jpg", cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR))

print("Grad-CAM saved → gradcam_result.jpg")


grad-worked: 0
